In [1]:
from bruker2nifti._metadata import BrukerMetadata
from pathlib import Path
from aind_data_schema.models.coordinates import Rotation3dTransform, Scale3dTransform, Translation3dTransform

import logging
import numpy as np

from datetime import datetime

In [2]:
log_file_name = "./log_files/log_" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".log"
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler(log_file_name, 'w', 'utf-8')
fh.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
# add the handlers to logger
logger.addHandler(fh)

In [3]:
path = Path("C:\\Users\\mae.moninghoff\\Documents\\GitHub\\aind-data-transfer\\scripts\\MRI_ingest\\MRI_files\\RawData-2023_07_21\\RAW\\DL_AI2.kX2")

metadata = BrukerMetadata(path)
metadata.parse_scans()
metadata.parse_subject()
subj = metadata.read_subject()
print(metadata.list_scans())
print(metadata.list_recons("7"))
recon = metadata.read_recon("7", "1")
print(recon)
print(subj)

#best to look at are acqp and metadata files
#reco / visupars can sometimes have useful data

print(metadata.subject_data)

['1', '2', '3', '4', '5', '6', '7']
['1']
{'reco': {'TITLE': 'Parameter List', 'JCAMPDX': 4.24, 'DATATYPE': 'Parameter Values', 'ORIGIN': 'Bruker BioSpin MRI GmbH', 'OWNER': 'dhlee', 'RECO_mode': 'FT_MODE', 'RecoAutoDerive': 'Yes', 'RECO_inp_order': 'NO_REORDERING', 'RECO_inp_size': array([400., 164., 130.]), 'RECO_ft_size': array([400., 166., 130.]), 'RECO_fov': array([2. , 1.4, 1.3]), 'RECO_size': array([200., 140., 130.]), 'RECO_offset': array([[100.],
       [ 13.],
       [  0.]]), 'RECO_regrid_mode': 'NO_REGRID', 'RECO_regrid_offset': 0.0, 'RECO_ramp_gap': 0.0, 'RECO_ramp_time': 0.0, 'RECO_ne_mode': 'NO_NAV_DATA', 'RECO_ne_dist': 'NeNoInterleave', 'RECO_ne_dens': 'NeCorrPerScan', 'RECO_ne_type': 'NAV_PHASE', 'RECO_ne_vals': array([[0., 0.]]), 'RECO_bc_mode': 'NO_BC NO_BC NO_BC', 'RECO_bc_start': array([300., 123.,  98.]), 'RECO_bc_len': array([100.,  41.,  32.]), 'RECO_dc_offset': array([0., 0.]), 'RECO_dc_divisor': 1.0, 'RECO_bc_coroff': array([0., 0., 0.]), 'RECO_qopts': 'CONJ_

In [4]:
# Checking Method Fields
cur_method = metadata.scan_data['7']['method']

# print(metadata.scan_data['7'].keys())
# print(self.cur_method.keys())
print(cur_method['Method'])
# print(self.cur_method['EchoTime'])
# print(self.cur_method['EffectiveTE'])
# print(self.cur_method['RareFactor'])
# print(self.cur_method['RepetitionTime'])
print(cur_method['SpatResol'])
print(type(cur_method['SpatResol'][0]))

RARE
[0.1 0.1 0.1]
<class 'numpy.float64'>


In [5]:
print(metadata.scan_data['7']['recons']['1']['visu_pars'])
cur_visu_pars = metadata.scan_data['7']['recons']['1']['visu_pars']
print(*cur_visu_pars['VisuCoreOrientation'])
print(cur_visu_pars['VisuCorePosition'])

scan7 = metadata.read_scan('7')


# print(scan7['recons']['1']['visu_pars'].keys())

{'TITLE': 'Parameter List', 'JCAMPDX': 4.24, 'DATATYPE': 'Parameter Values', 'ORIGIN': 'Bruker BioSpin MRI GmbH', 'OWNER': 'dhlee', 'VisuVersion': 1.0, 'VisuUid': '2.16.756.5.5.100.380435303.14105.1689969391.16', 'VisuCreator': 'ParaVision', 'VisuCreatorVersion': '5.1', 'VisuCreationDate': '11:27:33 21 Jul 2023', 'VisuCoreFrameCount': 1.0, 'VisuCoreDim': 3.0, 'VisuCoreSize': array([140., 200., 130.]), 'VisuCoreDimDesc': 'spatial spatial spatial', 'VisuCoreExtent': array([14., 20., 13.]), 'VisuCoreFrameThickness': 13.0, 'VisuCoreUnits': ['mm', 'mm', 'mm'], 'VisuCoreOrientation': array([[ 1.,  0.,  0.,  0.,  0., -1.,  0.,  1.,  0.]]), 'VisuCorePosition': array([[-6.1, -7. ,  7.9]]), 'VisuCoreDataMin': 49.0, 'VisuCoreDataMax': 32766.0, 'VisuCoreDataOffs': 0.0, 'VisuCoreDataSlope': 403.972115083361, 'VisuCoreFrameType': 'MAGNITUDE_IMAGE', 'VisuCoreWordType': '_16BIT_SGN_INT', 'VisuCoreByteOrder': 'littleEndian', 'VisuCoreDiskSliceOrder': 'disk_reverse_slice_order', 'VisuSubjectName': 'DL_A

In [6]:
from aind_data_schema.core.mri_session import MRIScan, MriScanSequence, ScanType, SubjectPosition
from decimal import Decimal
from aind_data_schema.models.units import MassUnit, TimeUnit

# method
    ##$Method=RARE
    ##$PVM_RareFactor=8,
    ##$PVM_EchoTime=0.01,
    ##$EffectiveTE=(1)
    # 10.6666667
    ##$PVM_RepetitionTime=500,
    ##$PVM_SpatResol=( 3 )
    # 0.1 0.1 0.1

# visu_pars
    ##$VisuCoreOrientation=( 1, 9 )
    # 1 0 0 0 0 -1 0 1 0
    ##$VisuCorePosition=( 1, 3 )
    # -6.1 -7 7.9

# subject
    ##$SUBJECT_position=SUBJ_POS_Supine

In [7]:
from bruker2nifti._metadata import BrukerMetadata
from pathlib import Path
from aind_data_schema.models.coordinates import Rotation3dTransform, Scale3dTransform, Translation3dTransform
from aind_data_schema.core.mri_session import MRIScan, MriSession, MriScanSequence, ScanType, SubjectPosition
from decimal import Decimal
from aind_data_schema.models.units import MassUnit, TimeUnit
from aind_data_schema.models.devices import Scanner, ScannerLocation, MagneticStrength
from datetime import datetime

import traceback


class MRILoader:
    def __init__(self, data_path):
        self.metadata = BrukerMetadata(path)
        self.metadata.parse_scans()
        self.metadata.parse_subject()
        self.n_scans = self.metadata.list_scans()


    def load_mri_session(self, experimenter: str, primary_scan_number: str, setup_scan_number: str, scan_location: ScannerLocation, magnet_strength: MagneticStrength) -> MRIScan:

        scans = []
        for scan in self.n_scans:
            scan_data = self.metadata.read_scan(scan)
            scan_type = "3D Scan"
            if scan == setup_scan_number:
                scan_type = "Set Up"
            primary_scan = False
            if scan == primary_scan_number:
                primary_scan = True
            scans.append(self.make_model_from_scan(scan, scan_type, primary_scan))

        return MriSession(

            subject_id="",
            session_start_time=datetime.now(), 
            session_end_time=datetime.now(),
            experimenter_full_name=experimenter, 
            protocol_id="",
            iacuc_protocol="",
            # animal_weight_prior=,
            # animal_weight_post=,
            # weight_unit=, 
            # anaesthesia=,
            mri_scanner=Scanner(
                name="test_scanner",
                scanner_location=scan_location,
                magnetic_strength=magnet_strength, 
                magnetic_strength_unit="T", 
            ),
            scans=scans,
            notes="none"
        )
    

    def make_model_from_scan(self, scan_index: str, scan_type, primary_scan: bool) -> MRIScan:
        print(f'loading scan {scan_index}')
        print(f'{self.metadata.read_scan(scan_index).keys()}')
        

        
        self.cur_visu_pars = self.metadata.scan_data[scan_index]['recons']['1']['visu_pars']
        self.cur_method = self.metadata.scan_data[scan_index]['method']


        print(self.metadata.scan_data[scan_index].keys())
        print(self.cur_method['EchoTime'])
        subj_pos = self.metadata.subject_data["SUBJECT_position"]
        if 'supine' in subj_pos.lower():
            subj_pos = 'Supine'
        elif 'prone' in subj_pos.lower():
            subj_pos = 'Prone'

        scan_sequence = MriScanSequence.OTHER
        if 'RARE' in self.cur_method['Method']:
            scan_sequence = MriScanSequence(self.cur_method['Method'])

        rare_factor = None
        if 'RareFactor' in self.cur_method.keys():
            rare_factor = self.cur_method['RareFactor']

        print(f'orientation: {self.cur_visu_pars["VisuCoreOrientation"]}')
        print(f'position: {self.cur_visu_pars["VisuCorePosition"]}')

        rotation=list(*self.cur_visu_pars['VisuCoreOrientation'])
        if isinstance(rotation, np.ndarray):
            rotation = rotation.tolist()
        translation=list(*self.cur_visu_pars['VisuCorePosition'])
        if isinstance(translation, np.ndarray):
            translation = translation.tolist()

        try:
            return MRIScan(
                scan_index=scan_index,
                scan_type=ScanType(scan_type), # set by scientists
                primary_scan=primary_scan, # set by scientists
                scan_sequence_type=scan_sequence, # method ##$Method=RARE,
                rare_factor=rare_factor, # method ##$PVM_RareFactor=8,
                echo_time=self.cur_method['EchoTime'], # method ##$PVM_EchoTime=0.01,
                effective_echo_time=Decimal(self.cur_method['EffectiveTE']), # method ##$EffectiveTE=(1)
                # echo_time_unit=TimeUnit(), # what do we want here?
                repetition_time=self.cur_method['RepetitionTime'], # method ##$PVM_RepetitionTime=500,
                # repetition_time_unit=TimeUnit(), # ditto
                vc_orientation=Rotation3dTransform(rotation=rotation),# visu_pars  ##$VisuCoreOrientation=( 1, 9 )
                vc_position=Translation3dTransform(translation=translation), # visu_pars ##$VisuCorePosition=( 1, 3 )
                subject_position=SubjectPosition(subj_pos), # subject ##$SUBJECT_position=SUBJ_POS_Supine,
                voxel_sizes=Scale3dTransform(scale=self.cur_method['SpatResol']), # method ##$PVM_SpatResol=( 3 )
                processing_steps=[],
                additional_scan_parameters={},
                notes="", # Where should we pull these?
            )      
        except Exception as e:
            print(traceback.format_exc())
            logging.error(f'Error loading scan {scan_index}: {e}') 

In [8]:
loader = MRILoader(path)
scan7 = loader.make_model_from_scan('5', '3D Scan', True)
print(scan7)

loading scan 5
dict_keys(['acqp', 'method', 'recons'])
dict_keys(['acqp', 'method', 'recons'])
5.33333333333333
orientation: [[ 1.  0.  0.  0.  0. -1.  0.  1.  0.]]
position: [[-6.1 -7.1  8.1]]
scan_index=5 scan_type='3D Scan' primary_scan=True scan_sequence_type='RARE' rare_factor=4 echo_time=Decimal('5.33333333333333') effective_echo_time=Decimal('10.6666666666666998253276688046753406524658203125') echo_time_unit=<TimeUnit.MS: 'millisecond'> repetition_time=Decimal('500.0') repetition_time_unit=<TimeUnit.MS: 'millisecond'> vc_orientation=Rotation3dTransform(type='rotation', rotation=[Decimal('1.0'), Decimal('0.0'), Decimal('0.0'), Decimal('0.0'), Decimal('0.0'), Decimal('-1.0'), Decimal('0.0'), Decimal('1.0'), Decimal('0.0')]) vc_position=Translation3dTransform(type='translation', translation=[Decimal('-6.1'), Decimal('-7.1'), Decimal('8.1')]) subject_position='Supine' voxel_sizes=Scale3dTransform(type='scale', scale=[Decimal('0.5'), Decimal('0.4375'), Decimal('0.52')]) processing_st

In [9]:
stuff = {
    'probes1': {
        'name': "name",
        'item_type': "type",
        'to_replace': "to_replace",
    },
    'probes2': {
        'name': "name",
        'item_type': "type",
        'to_replace': "to_replace",
    }  
}

for probe in stuff:
    print(probe)
    probe['to_replace'] = "replaced"

print(stuff)



probes1


TypeError: 'str' object does not support item assignment

In [ ]:
session = loader.load_mri_session(["Mae"], "7", "1", ScannerLocation.FRED_HUTCH, MagneticStrength.MRI_7T)

loading scan 1
dict_keys(['acqp', 'method', 'recons'])
dict_keys(['acqp', 'method', 'recons'])
3.42
orientation: [[ 1.  0.  0.  0.  1.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0. -1. -1.  0.  0.]
 [ 1.  0.  0.  0.  0. -1.  0.  1.  0.]]
position: [[-15. -15.   0.]
 [  0. -15.  15.]
 [-15.   0.  15.]]


TypeError: list expected at most 1 argument, got 3